## PropertyPilot Demo
## Conversational AI Capstone I
## 21 May 2024

In [58]:
# %pip list | grep google-cloud-aiplatform
# %pip list | grep google-api-core
# %pip install google-cloud-aiplatform==1.43.0
# %pip install google-api-core==2.17.1

In [59]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

import geopandas as gpd
from shapely.geometry import Point

import http.client
import json
import urllib.parse
from typing import Dict, Any
import pandas as pd
import folium

In [60]:
vertexai.init(project = "adsp-capstone-property-pilot", location = "us-central1")

## Model Setting - Gemini pro 1.0

In [61]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [62]:
# Test chat
prompt = "Hello."
print(get_chat_response(chat, prompt))

Hello! I am happy to hear from you. What can I help you with today?


## Settings

In [63]:
with open('/home/jupyter/zillow_api.txt', 'r') as file:
    zillow_api = file.read()

import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

headers = {
	"X-RapidAPI-Key": zillow_api,
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

with open('/home/jupyter/yelp_api.txt', 'r') as file:
    yelp_api = file.read()

yelp_api_key = yelp_api

## Import functions

In [64]:
import os

In [65]:
%run /home/jupyter/property_pilot/scripts/chat_response.py

In [66]:
%run /home/jupyter/property_pilot/scripts/generate_api_filter.py

In [67]:
%run /home/jupyter/property_pilot/scripts/fetch_property_info.py

In [68]:
%run /home/jupyter/property_pilot/scripts/yelp_property.py

In [69]:
%run /home/jupyter/property_pilot/scripts/neighborhood_rag.py

In [70]:
%run /home/jupyter/property_pilot/scripts/yelp_advisor.py

## Import data

In [71]:
# For RAG

# read csv
vector_store = pd.read_csv('/home/jupyter/property_pilot/data/vector_store.csv')
# embedding to numpy array
vector_store['embeddings'] = vector_store['embeddings'].apply(lambda x: np.array(list(map(float, x.split(',')))))

## Import prompts

In [72]:
os.chdir('/home/jupyter/property_pilot/prompts')

# prompt_classifier
file_path = 'instruction_classifier.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_classifier = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}") 
    
# prompt_apifilter
file_path = 'generate_prompt_apifilter_v2.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_apifilter = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")

# zillow_locations
file_path = 'zillow_locations.txt'
try:
    with open(file_path, 'r') as file:
        zillow_locations = file.read().splitlines()
        locations_string = ','.join(zillow_locations)
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_categories
file_path = 'yelp_categories.txt'
try:
    with open(file_path, 'r') as file:
        yelp_categories = file.read().splitlines()
        categories_string = ','.join(yelp_categories)
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# prompt_property(for final output)
file_path = 'generate_prompt_property.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_property = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# instruction_rag_neighborhood
file_path = 'instruction_rag_neighborhood.txt'
try:
    with open(file_path, 'r') as file:
        instruction_rag_neighborhood = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_advisor_instructions (for yelp local advisor)
file_path = 'yelp_advisor_instructions.txt'
try:
    with open(file_path, 'r') as file:
        yelp_advisor_instruction = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_output_instructions (for yelp local advisor)
file_path = 'yelp_output_instructions.txt'
try:
    with open(file_path, 'r') as file:
        yelp_output_instructions = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")

## Property Plot main function

In [73]:
def pp_chatbot(user_query):
    
    # User's quety classify
    prompt_classifier = generate_prompt_classifier(instruction_prompt_classifier, user_query)
    response_int = int(get_chat_response(chat, prompt_classifier))
    
    # if the user is looking for a property     
    if response_int == 1:

        # get prompt for api filter 
        prompt_apifilter = generate_prompt_apifilter_v2(instruction_prompt_apifilter, user_query, locations_string)

        # get api filter
        apifilter = get_chat_response(chat, prompt_apifilter)
        apifilter = extract_json_to_dict(apifilter)

        # get listings from zillow
        response = requests.get(url, headers = headers, params = apifilter)

        # get top 3 listings(sorted by newest)
        # Define fields to extract
        fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc", "longitude", "latitude"]
        top_properties = extract_properties(response, 3, fields)

        # get description of the properties
        top_properties = fetch_descriptions(top_properties)

        # get resoFacts(detail) of the properties
        keys_to_fetch_resoFacts = [
            'hasGarage', 'hasPetsAllowed', 'heating', 'cooling', 'flooring', 'appliances',
            'laundryFeatures', 'associationFee',
            'livingArea', 'taxAnnualAmount', 'parkingFeatures', 'stories'
        ]
        top_properties = fetch_resoFacts(top_properties, headers, keys_to_fetch_resoFacts)

        # get school info of the properties
        top_properties = fetch_schools(top_properties, headers)
        
        # get restaurant info from yelp api
        yelp_results = fetch_top_businesses_near_properties(yelp_api_key, top_properties, user_query, yelp_categories, chat)
        
        # extract restaurant info
        yelp_fields = ['business_name', 'categories_titles', 'rating', 'latitude', 'longitude', 'display_address', 'url']
        top_restaurants = extract_business_info(yelp_results, 2, yelp_fields)
        
        # merge top_restaurants with top_properties
        top_properties = merge_property_and_restaurant_info(top_properties, top_restaurants)

        # neighborhood boundaries shapefile
        neighborhood_boundaries_path = '/home/jupyter/property_pilot/data/neighborhood_boundaries/geo_export_825d7df4-a9cd-4cef-b3d7-2ec1adc30204.shp'
        # neighborhood description file
        neighborhood_info_path = '/home/jupyter/property_pilot/data/neighborhood_info/neighborhood_info_final.csv'

        # load the neighborhood boundaries and description file
        neighborhoods = load_neighborhood_boundaries(neighborhood_boundaries_path)
        neighborhoods_info = load_neighborhood_info(neighborhood_info_path)

        # get neighborhood details of the properties
        top_properties = fetch_neighborhood(top_properties, neighborhoods)
        top_properties = fetch_neighborhood_info(top_properties, neighborhoods_info)

        # create property map
        property_map = create_property_map(top_properties)
        
        # create property_restaurant map
        property_restaurant_map = create_property_restaurant_map(top_properties)

        # change format
        fields_desc = fields + ['description', 'resoFacts', 'schools', 'neighborhood', 'neighborhood_description', 'restaurant_1', 'restaurant_2']
        property_info = format_properties(top_properties, fields_desc)

        # prompt for final output from Gemini
        prompt_final = generate_prompt_property(instruction_prompt_property, user_query, property_info, str(fields))

        # get final response from Gemini
        print(get_chat_response(chat, prompt_final))
        display(property_restaurant_map)

        return top_properties

    elif response_int == 2:
        # print('The query is about restaurant. I will answer by using Yelp API Filter')
        yelp_prompt = generate_prompt_yelp_advisor(yelp_advisor_instruction, user_query, categories_string)
        
        # get yelp filter string
        yelp_filter_str = get_chat_response(chat, yelp_prompt)
        
        # get yelp filter in json format
        yelp_filter = extract_json_to_dict(yelp_filter_str)
        
        # get response from yelp api
        yelp_advisor_results = yelp_advisor(yelp_api_key, yelp_filter)
        
        # create yelp restaurant map
        restaurant_map = create_yelp_restaurant_map(yelp_advisor_results)
        
        # generate yelp output prompt
        yelp_output_prompt = final_output_yelp_advisor(yelp_output_instructions, user_query, yelp_advisor_results)
        
        # get final response from Gemini
        print(get_chat_response(chat, yelp_output_prompt))
        display(restaurant_map)
        
    elif response_int == 3:
        # print('The query is about neighborhood. I will answer by using RAG')
        context = get_context(question = user_query, vector_store=vector_store, num_docs=10)
        prompt_rag_neighborhood = generate_prompt_rag_neighborhood(instruction_rag_neighborhood, context, user_query)
        print(get_chat_response(chat, prompt_rag_neighborhood))
        # return context
    else:
        print(get_chat_response(chat, user_query))

## Use case 1
### property search - detail info

In [76]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "I am looking to purchase a house in Gold Coast, Chicago. I would like to have a minimum 3 bedrooms and 3 bathrooms. I want to pay less than $3,000,000."
top_properties = pp_chatbot(user_query)

## Properties in Gold Coast that meet your criteria:

Here are three properties in Gold Coast that match your criteria of having at least 3 bedrooms and 3 bathrooms, and being priced under $3,000,000:

**1. 1358 N State Pkwy, Chicago, IL 60610**

* **Price:** $2,950,000
* **Bedrooms:** 5
* **Bathrooms:** 6
* **Property Type:** Single Family
* **Description:** This beautifully renovated, turn-key Gold Coast residence offers 5 bedrooms with ensuite baths, a sun-filled living room, a formal dining room, a half bath, and a cheerful west-facing kitchen featuring a six-burner Viking range, Miele dishwasher and refrigerator, Subzero freezers, Scotsman icemaker, and a separate walk-in pantry with refrigerator/freezer. Upstairs, the primary suite has a working gas fireplace, impressive east-facing windows, and a luxurious bath with separate tub and shower, double vanity, heated floor, and extra storage. A large walk-in closet completes the space. This floor also includes a laundry room and a la

In [77]:
top_properties

[{'propertyType': 'SINGLE_FAMILY',
  'address': '1358 N State Pkwy, Chicago, IL 60610',
  'price': 2950000,
  'bedrooms': 5,
  'bathrooms': 6,
  'detailUrl': 'https://www.zillow.com/homedetails/1358-N-State-Pkwy-Chicago-IL-60610/3856630_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/038119f7a00e1988b42cba467dc87c34-p_e.jpg',
  'longitude': -87.62914,
  'latitude': 41.90752,
  'description': 'Expect moments of sheer delight when you enter this beautifully renovated, turn-key Gold Coast residence. Measuring nearly 5000 sq ft, this home offers 5 bedrooms w/ensuite baths, sun-filled living room with wood-burning fireplace, formal dining room, half bath, and a cheerful west-facing kitchen featuring a six-burner Viking range plus additional wall oven, Miele dishwasher and refrigerator, Subzero freezers, Scotsman icemaker, and a separate walk-in pantry with refrigerator/freezer. Ascend to the 2nd floor to the primary suite, expertly appointed with a working gas fireplace, impressive 

In [79]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Can you tell me more about the house on Cedar St?"
print(get_chat_response(chat, user_query))

## Additional details about the house on 50 E Cedar St, Chicago, IL 60611:

**Key Features:**

* **Architecturally significant:** This timeless Romanesque-designed single-family home was built in 1892 by the prominent architect L. Gustav Hallberg.
* **Spacious:** The house boasts 5,000 square feet of living space, 6 bedrooms, and 6 bathrooms, providing ample room for a large family or those who enjoy entertaining.
* **Unique outdoor space:** The property features a large, professionally landscaped backyard with a patio, pergola, and gazebo, creating a private oasis in the heart of the city.
* **Luxurious details:** The home showcases original architectural details like wood paneling, fireplaces, a grand staircase, and a gold-leaf ceiling in the dining room.
* **Modern amenities:** The kitchen features granite countertops, a spacious island, and top-of-the-line appliances, while the bathrooms offer modern fixtures and finishes.
* **Ideal location:** Located on tree-lined Cedar Street, t

In [81]:
user_query = "Which neighborhood is the third house located in? Tell me more about that neighborhood."
print(get_chat_response(chat, user_query))


### The house on 50 E Cedar St is located in the Gold Coast neighborhood of Chicago. 

## More about the Gold Coast neighborhood:

The Gold Coast is celebrated for its:

* **Luxury:** Historic mansions, high-end boutiques, and renowned restaurants line the streets, offering a taste of the high life.
* **Architectural beauty:**  The neighborhood boasts stunning architecture, including the former Playboy Mansion and the elegant Astor Street District. 
* **Upscale experiences:**  From fine dining and vibrant nightlife to sophisticated shopping, the Gold Coast caters to those seeking a lavish urban experience. 
* **Proximity to attractions:** The lakefront, museums, and cultural institutions are all within easy reach, providing residents with endless entertainment options.



In [82]:
user_query = "What are some good schools nearby the third house?"
print(get_chat_response(chat, user_query))

##  Schools near 50 E Cedar St, Chicago, IL 60611:

**Elementary Schools:**

* **Ogden Elementary School:** Public, Rating: 5, Distance: 0.2 miles 
    * Ogden is a highly regarded public school with a diverse student body and a strong academic record. It offers various programs and extracurricular activities, fostering a well-rounded educational experience.
* **St. Chrysostom's Day School:** Private, Pre-K to 8th Grade, Rating: 4.8, Distance: 0.4 miles
    * This private school offers a faith-based education with small class sizes and individualized attention. It emphasizes academic excellence while fostering spiritual and moral development.

**High Schools:**

* **Lincoln Park High School:** Public, Rating: 9, Distance: 1.5 miles
    * Lincoln Park High School is one of the top public schools in the city, consistently ranking in the top 10% nationally. The school offers a comprehensive curriculum, advanced placement courses, and a variety of extracurricular activities. It has a diver

In [83]:
user_query = "What restaurants are nearby the house? Please provide me with the yelp link"
print(get_chat_response(chat, user_query))

## Restaurants near 50 E Cedar St, Chicago, IL 60611:

**Location:**

The house is situated in the heart of the Gold Coast, surrounded by a plethora of culinary options. Here are some restaurants nearby:

**High-End Dining:**

* **Bavette's Bar & Boeuf:** Classic steakhouse known for its dry-aged steaks and elegant ambiance. (0.2 miles) https://www.yelp.com/biz/bavettes-bar-and-boeuf-chicago
* **Gibson's Steakhouse:** Another upscale steakhouse with a lively atmosphere and a wide selection of wines. (0.3 miles) https://www.yelp.com/biz/gibsons-steakhouse-chicago-2
* **RPM Italian:** Modern Italian restaurant with a sophisticated ambiance and delicious pasta dishes. (0.4 miles) https://www.yelp.com/biz/rpm-italian-chicago-3

**Casual Options:**

* **Gold Coast Market:** Deli with a variety of prepared foods, sandwiches, and drinks. (0.1 miles) https://www.yelp.com/biz/gold-coast-market-chicago
* **Little Goat Diner:** Quirky diner serving American comfort food with a modern twist. (0.4 

In [84]:
#top_properties

## Use case 2
### Initial query: Restaurant info

In [29]:
chat = model.start_chat()
# Test chat
# prompt = "Hello."
# print(get_chat_response(chat, prompt))

In [33]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "What are the top rated Indian restaurants on the north side of chicago?"
pp_chatbot(user_query)

Absolutely! Based on your query, here are some of the top-rated Indian restaurants on the north side of Chicago:

1. **Art of Indian Cuisine (4.9 stars):** This highly-rated gem offers authentic Indian dishes with a focus on fresh, seasonal ingredients. Reviewers rave about the flavorful curries, delectable tandoori dishes, and friendly service.

2. **Bhoomi Modern Indian Grill (4.7 stars):** This modern eatery boasts a sophisticated atmosphere and a menu featuring innovative takes on classic Indian cuisine. Enjoy plates like black truffle naan, lamb chops, and creative vegetarian options.

3. **Krishna Lunch (4.7 stars):** This vegetarian lunch spot is a haven for those seeking flavorful and affordable Indian fare. The menu changes daily, offering a variety of curries, rice dishes, and fresh salads.

4. **Apna Indian Grill (4.6 stars):** This casual eatery offers a wide range of classic Indian dishes, from curries and biryanis to tandoori specialties. The portions are generous, and th

In [34]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Which of these restaurants are budget-friendly?"
print(get_chat_response(chat, user_query))

## Budget-Friendly Indian Restaurants on the North Side

While all the restaurants I recommended have reasonable prices for their quality, here's a breakdown based on online information and reviews:

**Most Budget-Friendly:**

* **Krishna Lunch:** This vegetarian spot is known for its incredibly affordable lunch buffets, starting at around $10. You can also get individual dishes for under $10.
* **Apna Indian Grill:** This casual eatery offers generous portions at reasonable prices. Most dishes are priced between $10-$15.

**Moderately Priced:**

* **Art of Indian Cuisine:** While offering high-quality ingredients, this restaurant has a slightly higher price range. Most entrees fall between $15-$25.
* **Bhoomi Modern Indian Grill:** This modern restaurant has a contemporary atmosphere and slightly higher price point than the others. Entrees typically range from $20-$35.
* **Indian Paradise:** This family-owned restaurant offers a good value for its authentic North Indian dishes. Most e

In [36]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Could you provide me a link to Bhoomi Modern Indian Grill's menu?"
print(get_chat_response(chat, user_query))

## Bhoomi Modern Indian Grill Menu

Here's the link to Bhoomi Modern Indian Grill's menu: https://www.eatbhoomi.com/menu

Please note that online menus may not always be completely up-to-date, so it's always a good idea to call the restaurant directly for the most accurate information.





## Use case 3
### Initial query: Neighborhood info

In [39]:
chat = model.start_chat()

In [40]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "What is the Hyde Park in Chicago like?"
pp_chatbot(user_query)

## Hyde Park in Chicago: A Detailed Look

Hyde Park is a vibrant neighborhood on the south side of Chicago, offering a unique blend of history, culture, and natural beauty. Here's a breakdown of its key features:

**Things to Do**:

* **Museums & Culture:** Hyde Park is home to the renowned University of Chicago and its associated museums, including the Oriental Institute Museum and the Smart Museum of Art. The neighborhood also boasts the iconic Museum of Science and Industry, a family-friendly destination with interactive exhibits and educational programs.
* **Green Spaces:** Relax and unwind in the expansive Jackson Park, offering stunning views of Lake Michigan, walking trails, and the beautiful landscaped gardens. The park also houses the Museum of Science and Industry and the Osaka Garden, a traditional Japanese garden.
* **Diverse Dining:** Hyde Park caters to a variety of palates with its wide selection of restaurants. From casual eateries and cafes to upscale dining options, t

In [41]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Tell me more about the neighborhood?"
print(get_chat_response(chat, user_query))

## Delving Deeper into Hyde Park:

While I've covered the main highlights of Hyde Park, there's always more to explore in this dynamic neighborhood. Here are some additional details:

**History:**

* Founded in 1853, Hyde Park boasts a rich history. Its development as a planned community was envisioned by Paul Cornell, who aimed to create a picturesque suburb with spacious lots, tree-lined streets, and a commitment to education and culture.
* The arrival of the University of Chicago in 1892 significantly shaped the neighborhood. Its iconic campus architecture and academic reputation attracted intellectuals, artists, and scholars, contributing to Hyde Park's vibrant intellectual atmosphere.
* Throughout the 20th century, Hyde Park played a pivotal role in social movements like the Civil Rights movement. Its diverse population and progressive values fueled activism and social change initiatives.

**Architecture:**

* Hyde Park showcases a diverse array of architectural styles, reflecting